<h1 style="text-align: center; font-weight:bold;"> Higgs-Suche mit Python </h1>

> von Kai Adamowicz und Philipp Nattland, basierend auf der CMS Open Data Analyse *"Analysis of Higgs boson decays to four leptons using data and simulation of events at the CMS detector from 2012 using ROOT's RDataFrame"* von Stefan Wunsch auf [https://opendata.cern.ch/record/12360](https://opendata.cern.ch/record/12360)

Das Standardmodel der Teilchenphysik ist ein wohlerprobtes Model um die Interaktionen von Teilchen vorrauszusagen. Es wurde jedoch nicht an einem Tag erschaffen sondern über viele Jahzehnte durch unzählige Experimente sorgfältig Stein um Stein aufgebaut. Viele Fragen und Rätsel um das Standardmodel blieben jedoch eine lange Zeit unbeantwortet. So zum Beispiel die Frage warum das Photon - das Austauschteilchen der elektromagnetischen Wechselwirkung - masselos ist, während das W- und Z-Boson - die Austauschteilchen der schwachen Wechselwirkung - massiv sind. 1964 stellte Peter Higgs (sowie mehrere weitere Wissenschaftler) eine Theorie vor, welche diese Inkonsistenz erklären kann ohne dem Rest des Standardmodels zu widersprechen. Diese Theorie implizierte jedoch die Existenz eines neuen, bisher unbekannten Teilchens - das **Higgs** Boson. Eine lange Zeit blieb es unentdeckt, das LEP Experiment (Vorgänger des LHC Experiments) endete mit dem Ergebnis, dass das Higgs Boson eine Masse von über 114 GeV haben muss - denn schwerere Teilchen konnte das LEP aus technischen Gründen nicht erzeugen. Das nachfolgende LHC Experiment ist in der Lage Teilchen mit deutlich höheren Massen zu erzeugen und sollte das Mysterium um das Higgs Boson endlich lösen.

Im Sommer 2012 verkündeten die **CMS** und **ATLAS** Kollaborationen die Entdeckung eines neuen Teilchens am CERN LHC. Die Eigenschaften des Teilchens stimmten mit der Theorie von Peter Higgs und seinen Kollegen überein. Das Higgs Boson wurde gefunden - bei einer Masse von ca. 125 GeV - nur knapp ausserhalb der maximalen Reichweite des LEP Experiments. Im folgenden Jupyter Notebook werden wir die Schritte der CMS Wissenschaftler nachvollziehen und das Higgs Boson im $\text{H}\to\text{ZZ}\to4\ell$ Kanal erneut "entdecken". Dafür verwenden wir die gleichen Daten, die 2012 zur Entdeckung geführt haben.
> **Good to know:** Zur Entdeckung des Higgs Bosons trugen auch andere Zerfallskanäle, wie z.B. $\text{H}\to\gamma\gamma$ bei (Das $\gamma$ bezeichnet hier das Photon).

## Der Datensatz
<hr>

Wie bereits erwähnt werden wir die originalen Datensätze verwenden, die 2012 zur Entdeckung des Higgs Bosons beigetragen haben. Diese wurden von der CMS Kollaboration öffentlich gemacht und können auf der CERN Open Data Plattform gefunden werden. Wie ihr in der Einführung gelernt habt, nimmt das CMS Experiment unvorstellbar große Datenmengen auf, daher ist es unpraktisch **alle** aufgenommen Daten hier zu analysieren. Stattdessen wurde eine Vorauswahl an Daten getätigt, um die Dateigröße zu reduzieren. Da wir an dem $\text{H}\to\text{ZZ}\to4\ell$ Prozess interessiert sind, ist das Hauptkriterium, auf Basis dessen wir diese Vorauswahl getätigt haben, dass in jedem Event mindestens 4 Leptonen (also Elektronen oder Myonen) enthalten sein müssen. Auf diese Weise konnten wir die Datenmenge um ein Vielfaches reduzieren und gleichzeitig einen großen Teil der $\text{H}\to\text{ZZ}\to4\ell$ Events beibehalten.

Zunächst importieren wir verschiedene Module, welche wir für die Analyse benötigen. Diese enthalten nützliche *Funktionen* (siehe Python-Übung), welchen wir lediglich ein paar Parameter übergeben müssen, um eine bestimmte Aufgabe zu lösen. Um die Funktionen aus den Bibliotheken zu laden, führe einfach den unteren Codeblock aus.

In [ ]:
import awkward as ak   # Wichtiges Modul um große Arrays zu bearbeiten
from helpers import *  # Alle Funktionen aus unserem selbtgeschriebenen "helpers" Modul

Mit den geladenen Funktionen können wir nun die Daten einlesen. Die entsprechenden Dateien sind im Ordner `data_files` gespeichert. Mit dem Folgenden Befehl werden alle Dateien eingelesen und dem *dict* `data` hinzugefügt.  

In [ ]:
data = load_data("data_files")

Wenn wir uns dieses *dict* jetzt genauer anschauen, sehen wir, dass in diesem die für uns relevanten Objekte, nämlich die Leptonen gespeichert sind, mit ihren für uns relevanten kinemantischen Eigenschaften $p_\text{T}$, $\eta$, $\phi$, Ladung und Masse.
> **Good to know:** Die meisten Events, die vom CMS aufgenommen werden enthalten auch eine Vielzahl an **Jets**. Da diese für unsere Analyse irrelevant sind haben wir sie entfernt um Platz zu sparen.

In [ ]:
print(data.keys())
print(data["muons"])
print(data["muons"]["pt"])

## Definition der Objekte
<hr>

In `data` sind jetzt all unsere Events enhalten. Da für unsere Analyse die Leptonen so wichtig sind, wollen wir das diese besonders gut rekonstruiert sind. Um das zu garantieren werden wir im Folgenden nur noch Leptonen in Betracht ziehen, die die Folgenden Bedingungen erfüllen:

- Der transversale Impuls $p_\text{T}$ von Myonen bzw. Elektronen ist größer als $5\,\text{GeV}$ bzw.  $7\,\text{GeV}$
- Die Leptonen sind innerhalb des Spurdetektors, also $|\eta|<2.4$

Die erste Bedingung verhindert, dass Elektronen und Myonen "zu niedrigen" Energien in unserer Analyse beitragen, da der CMS Detektor Teilchen mit zu niedrigen Energien nicht genau genug vermessen kann. Mit der zweiten Bedingung entfernen wir alle Leptonen, die zu nah "am Rand" des Detektors gemessen wurden, denn in dieser Randregion ist der Detektor ebenfalls nicht sehr genau.

Wir können zunächst ein Histogram mit den Transversalimpulsen und Pseudorapiditäten der Myonen und Elektronen erstellen.

In [ ]:
muons = data["muons"]
electrons = data["electrons"]

plot_kinematics(muons, electrons, label = ["Muons", "Electrons"])

Jetzt können wir die Bedingungen an die Objekte, die wir oben definiert haben, implementieren. Dazu nutzen wir die relationalen Operatoren  `>`  und  `<`, die ihr im Pythontutorial kennengelernt habt:

In [ ]:
passes_pt = (muons["pt"] > 5)
passes_eta = abs(muons["eta"]) < 2.4

Von jetzt an, wollen wir uns nur noch die Myonen angucken, die diese Anforderungen erfüllen. Da beide Anforderungen erfüllt sein sollen, nutzen wir den "und" operator `&`. Wir können zusätzlich auch die Anzahl aller Myonen in unserem Datensatz vor und nach dem Cut ausgeben lassen um zu sehen, wie viel Myonen wir verlieren.

In [ ]:
print("Anzahl Myonen vor dem Cut:", ak.sum(ak.num(muons)))

muons = muons[passes_pt & passes_eta]

print("Anzahl Myonen nach dem Cut:", ak.sum(ak.num(muons)))

Wenn wir uns nun nochmal die Histogramme anschauen sehen wir, dass es keine Myonen mit einem $p_\text{T}$ unter $5 \ \text{GeV}$ und keine Myonen mit $|\eta| > 2.4$ mehr gibt:

In [ ]:
plot_kinematics(muons, electrons, label = ["Muons", "Electrons"])

Nun müssen aber nicht nur die Myonen, sondern auch die Elektronen $p_\text{T}$ und $\eta$ Anforderungen erfüllen (siehe oben). Versucht im Folgenden diese zu implementieren. Um zu überprüfen ob alles funktioniert hat, gebt euch die Anzahl der Elektronen vor und nach Cut aus. Schaut euch ebenfalls die Histogramme nochmals an!

In [ ]:
# TODO: Implementiert hier die Elektron Bedingungen equivalent zu dem, was wir oben für die Myonen gemacht haben!
# TODO: Wie viele Elektronen waren vor und nach dem Cut im Datensatz vorhanden?

electrons = 
n_electrons = ak.num(electrons)
print("Anzahl Elektronen vor dem Cut:", ak.sum(n_electrons))

passes_pt = 
passes_eta = 
electrons = electrons[passes_pt & passes_eta]
n_electrons = ak.num(electrons)
print("Anzahl Elektronen nach dem Cut:", ak.sum(n_electrons))

plot_kinematics(muons, electrons, label = ["Muons", "Electrons"])

Spielt gerne mit den Cuts herum! Erhöht die Anforderungen, gebt euch die kinematischen Eigenschaften aus und überprüft ob die Cuts funktioniert haben!

## Selektion der Events
<hr>

Nachdem wir nun unsere Myonen und Elektronen gefiltert haben, können wir uns die kombinierte Masse der vier Leptonen $m_{4\ell}$ anschauen. Das entsprechende Histogramm kann mit der Funktion `plot_m4l()` erstellt werden, wobei die Elektronen und Myonen als Argumente übergeben werden müssen:

In [ ]:
plot_m4l(electrons, muons)

Wir sehen einen markanten Peak, der ber einer Masse von ungefähr 90 GeV liegt. Das dazugehörige Teilchen kennen wir bereits - es ist das Z-Boson (eines der Austauschteilchen der schwachen Wechselwirkung). Doch vom Higgs-Boson, welches wir bei einer Masse von 125 GeV erwarten, fehlt jede Spur. Der Grund hierfür sind **Hintergrundevents**, in anderen Worten, Prozesse die - wie unser Signal - ebenfalls vier Leptonen erzeugen und somit in unserem Histogramm gelandet sind. Wenn wir zu viele dieser Hintergrundevents in unser Histogramm lassen, können wir den Peak des Higgs-Bosons nicht erkennen. In diesem Kapitel versuchen wir die Events zu filtern um Hintergrundevents und $\text{H}\to\text{ZZ}\to4\ell$ Events besser zu trennen. Schauen wir uns zunächst die folgenden Events an:

In [ ]:
event_idx = 30
print("Event #31")
print("Myon Ladung:", muons["charge"][event_idx])
print("Elektron Ladung:", electrons["charge"][event_idx])

event_idx = 0
print("\nEvent #1")
print("Myon Ladung:", muons["charge"][event_idx])
print("Elektron Ladung:", electrons["charge"][event_idx]) 

Um zu verstehen, was an diesen Events "falsch" ist, werfen wir nochmals einen Blick auf das Feynman-Diagramm unseres $\text{H}\to\text{ZZ}\to4\ell$ Signalprozesses:

<img src="feynman.png" alt="Feynman_HtoZZto4l" width="300"> 

- In Event **#31** finden wir **drei** Myonen und **ein** Elektron. Allerdings kann ein Z-Boson nur in Leptonen des gleichen Typs zerfallen, also entweder $Z\to e^{+}e^{-}$ oder $Z\to \mu^{+}\mu^{-}$. Es ist also unmöglich eine ungerade Anzahl and Myonen bzw. Elektronen vorzufinden.

- In Event **#1** finden wir zwei Myonen und zwei Elektronen. Jedoch sind die Ladungen der Elektronen gleich. Da das Z-Boson elektrisch neutral ist müssen sich die Ladungen der Zerfallsleptonen aufheben. Das ist hier jedoch nicht der Fall.

Damit können die beiden obigen Events keine Signalevents sein. In den folgenden Unterkapiteln werden wir Filter einbauen, die **alle** Events entfernen, die ungerade Elektron- bzw. Myonzahlen aufweisen, sowie gegen die Ladungserhaltung verstoßen.

### Anzahl der Elektronen und Myonen
Als ersten Selektionsschritt können wir nun für alle Events fordern, dass es eine gerade Anzahl an Elektronen und Myonen geben muss. Unter dieser Voraussetzung, gibt es für uns drei verschiedene Fälle zu betrachten. Der erste Fall wäre, dass es genau 4 Elektronen gibt und keine Myonen. Welche weiteren zwei Fälle gibt es?

In **Python** kann man Forderungen an die Anzahl bestimmter Objekte mit der Funktion `ak.num()` machen (num ist hier kurz für "number", also Anzahl). Im Folgenden ist der Fall 4 Elektronen und 0 Myonen bespielhaft implementiert. Ergänzt diesen Fall um die beiden anderen Fälle, die ihr euch überlegt habt.

In [ ]:
fall_1 = (ak.num(muons) == 0) & (ak.num(electrons) == 4)
# TODO: Implementiert hier Fall 2 und Fall 3
fall_2 = 
fall_3 = 

Nun verlangen wir von allen events, dass sie einen dieser Fälle erfüllen, dafür benutzen wir die "oder" funktion in python`|`:

In [ ]:
passes_num = fall_1 | fall_2 | fall_3
electrons = electrons[passes_num]
muons = muons[passes_num]

Jetzt können wir uns erneut die Verteilung der $m_{4\ell}$ ansehen:

In [ ]:
plot_m4l(electrons, muons)

Viel hat sich nicht geändert, es gibt aber noch einige Cuts die wir implementieren können.

### Ladung der Leptonen
Als zweiten Selektrionsschritt verlangen wir dass die Summe der Elektronladungen und die Summe der Myonladungen beide null sind. Für Elektronen kann dies wie folgt implementiert werden:

In [ ]:
sum_charge_electron = ak.sum(electrons["charge"], axis = 1)
has_0Q_electrons = (sum_charge_electron == 0)

Hierbei berechnet die Funktion `ak.sum()` die summe der Ladung der Elektronen (`electrons["charge"]`), wobei `axis=1` bewirkt, dass das für jedes Event einzeln gemacht wird und nicht die Ladung über alle Events summiert wird. Dann fordern wir, dass diese Summe genau gleich null ist (`== 0`). Macht im folgenden Block das Gleiche für die Myonen!

In [ ]:
# TODO: Implementiert die Ladungsanforderung für die Myonen!
sum_charge_muon = 
has_0Q_muons =

Diese Forderungen wenden wir jetzt auf alle Events an, wobei immer **beides** gelten muss, Ladungssumme der Elektronen **und** (`&`) der Myonen ist null:

In [ ]:
passes_0Q = has_0Q_electrons & has_0Q_muons
electrons = electrons[passes_0Q]
muons = muons[passes_0Q]

Nachdem wir diese zweite Bedingung eingebaut haben, können wir uns wieder die Verteilung $m_{4\ell}$ anzeigen lassen. Macht dies im unteren Codeblock!

In [ ]:
# TODO: Lasst euch die m_4l verteilung anzeigen!

Jetzt könnt ihr womöglich schon einen sehr kleinen Peak bei 125 GeV sehen. Um diesen noch deutlicher sichtbar zu machen, wenden wir nun eine Bedingung an, die den Untergrund noch weiter reduzieren soll.

### Z Massen Fenster
Da die vier Leptonen in unserem Endzustand aus zwei Z-Bosonen entspringen, können wir uns jetzt die Leptonen angucken, die am ehesten "zusammen gehören", also vom gleichen Z abstammen. Für diese sollte nämlich gelten, dass die invariante Masse ungefähr der Z Masse ($m_Z = 91\,\text{GeV}$) enspricht. Außerdem ergibt es Sinn zu fordern, dass die Leptonen in einem solchen Paar, nicht zu nah aneinander sind, damit wir sicher sein können, dass beide tatsächlich aus dem ursprünglichen Prozess kommen und nicht eines erst später abgestrahlt wurde.

Etwas konkreter, rekonstruieren wir mit einem Algorithmus (implementiert als `get_Z_candidates()`) die zwei Leptonpaare die am ehesten zusammen gehören und berechnen deren kombinierte Masse und Abstand. Für die invariante Masse von Paar 1 soll gelten $40\,\text{GeV} < $ `m_Z1` $<120\,\text{GeV}$, für Paar 2 soll gelten $12\,\text{GeV} < $ `m_Z2` $ <120\,\text{GeV}$. Für die Abstände der Leptonen von Paar 1 und Paar 2 soll gelten `dR1`$>0.02$, bzw. `dR2` $>0.02$.

Im folgenden Block ist eine Implementierung angefangen. Versucht diese zu vervollständigen!

> **Good to know:** Die Funktion `get_Z_candidates()` geht alle möglichen (und erlaubten) Kombination von Leptonen durch und wählt das Leptonpaar, dessen Masse am nähesten am Z-Boson liegt als Paar #1 aus. Das übrig gebliebene Paar ist dann Paar #2. 

In [ ]:
### TODO: Implementiert die Cuts
### Z Fenster
mZ_1, mZ_2, dR1, dR2 = get_Z_candidates(muons, electrons)
passes_Z1 = 
passes_Z2 = 

### dR cut
passes_dRcut =

# Beide zusammenführen
passes_Zcuts =
electrons = electrons[passes_Zcuts]
muons = muons[passes_Zcuts]

Und nun könnt ihr euch $m_{4\ell}$ wieder anzeigen lassen:

In [ ]:
plot_m4l(electrons, muons)

Probiert mal die Fenstergröße um die Z-Masse, sowie den dR-cut zu verändern und guckt, wie es sich auf die $m_{4\ell}$-Verteilung auswirkt. 

### $p_\text{T}$ Bedingung im 2e2$\mu$ Kanal

Im 2e2 $\mu$ kanal muss aus "technischen" Gründen (bei Interesse fragt gerne die Betreuer*innen) ein zusätzlicher $p_\text{T}$-cut gemacht werden. Dieser ist implementiert als `get_2e2mu_pT_cut()`:

In [ ]:
passes_pt_2e2mu = get_e_mu_pT_cut(electrons, muons)
electrons = electrons[passes_pt_2e2mu]
muons = muons[passes_pt_2e2mu]

Diese letzte Anforderung sollte aber keinen großen Unterschied machen:

In [ ]:
plot_m4l(electrons, muons)

Bei der Betrachtung unseres Histogramms sticht nun ein Datenpunkt sehr stark hervor. Bei einer Masse von ca. 125 GeV zählen wir 10 Events, wobei in den umliegenden Bins in der Regel nur 1-2 Events enthalten sind. Mit unserem heutigen Wissen ist dieser Ausreißer keineswegs überraschend, denn wir würden bei dieser Masse das Signal des Higgs Bosons erwarten. Wie statistisch Signifikant ist dieser Peak? Könnte es ein statistischer Ausreißer sein? Diskutiert untereinander!

## Vergleich mit Simulation
<hr>

Im nächsten Schritt wollen wir unser $m_{4\ell}$-Histogramm mit **simulierten Daten** vergleichen. Simulationen spielen eine sehr wichtige Rolle in der Teilchenphysik, da sie uns erlauben gemessene Daten mit der Erwartung des Standardmodels zu vergleichen. Auf diese Weise können wir verifizieren, dass das Standardmodel (auf dem die Simulationen schließlich basieren) die realen Daten beschreibt und neue, unentdeckte Physik leichter erkennen.

Zwei physikalische Prozess spielen hier eine besondere Rolle: Zum einen der Hintergrund, der fast komplett aus sogenannten "Drell-Yan"-Ereignissen besteht (in unserem Fall hauptsächlich $Z/\gamma \to ZZ \to 4\ell$) und zum anderen aus unserem Signalprozess $\text{H}\to\text{ZZ}\to4\ell$. 

Laden wir zunächst die Simulation.

In [ ]:
sim = load_sim("sim_files")
print(sim.keys())
print(sim["ZZTo4e"].keys())

Wir sehen nun dass `sim` im Gegensatz zu `data` nicht nur aus Myonen und Elektronen besteht sondern zunächst in die verschiedenen Subprozesse eingeteilt ist. Als nächstes müssen wir all diese Subprozesse durch die gleichen Lepton- und Eventselektionen schicken, wie die originalen Daten - nur so können wir einen wirklichen Vergleich ziehen!

Im unteren Codeblock ist eine **Schleife** implementiert, die alle Anweisungen innerhalb der Schleife für alle Subprozesse durchführt. Ergänzt die fehlenden Anweisungen, wie ihr es bereits oben gemacht habt. Ihr könnt selbstverständlich eure Lösungen kopieren und wieder einfügen.

In [ ]:
electrons_by_process = {}
muons_by_process = {}

for process in sim.keys():

    sim_electrons = sim[process]["electrons"]
    sim_muons = sim[process]["muons"]

    ### --- Auswahl der Objekte ------------------------------------------------------------------------------------
    passes_pt = (sim_muons["pt"] > 5)
    passes_eta = abs(sim_muons["eta"]) < 2.4
    sim_muons = sim_muons[passes_pt & passes_eta]
    
    passes_pt = 
    passes_eta = 
    sim_electrons = 

    

    ### --- Anzahl der Elektronen und Myonen -----------------------------------------------------------------------
    fall_1 = (ak.num(sim_muons) == 0) & (ak.num(sim_electrons) == 4)
    fall_2 =
    fall_3 =
    passes_num = fall_1 | fall_2 | fall_3
    sim_electrons = sim_electrons[passes_num]
    sim_muons = sim_muons[passes_num]


    
    ### --- Ladung der Leptonen ------------------------------------------------------------------------------------
    sum_charge_electron = ak.sum(sim_electrons["charge"], axis = 1)
    has_0Q_electrons = (sum_charge_electron == 0)
    sum_charge_muon = 
    has_0Q_muons = 
    passes_0Q = has_0Q_electrons & has_0Q_muons
    sim_electrons = sim_electrons[passes_0Q]
    sim_muons = sim_muons[passes_0Q]


    
    ### --- Z Massenfenster -----------------------------------------------------------------------------------------
    mZ_1, mZ_2, dR1, dR2 = get_Z_candidates(sim_muons, sim_electrons)
    passes_Z1 = 
    passes_Z2 = 

    ### dR cut
    passes_dRcut =

    # Beide zusammenführen
    passes_Zcuts = 
    sim_electrons = sim_electrons[passes_Zcuts]
    sim_muons = sim_muons[passes_Zcuts]



    ### pT Bedingung im 2e2mu Kanal
    passes_pt_2e2mu = get_e_mu_pT_cut(sim_electrons, sim_muons)
    sim_electrons = sim_electrons[passes_pt_2e2mu]
    sim_muons = sim_muons[passes_pt_2e2mu]
    

    electrons_by_process[process] = sim_electrons
    muons_by_process[process] = sim_muons

Abschließend plotten wir wieder ein $m_{4\ell}$-Histogramm, diesmal aber mit einer anderen Funktion, da wir auch die Simulationen mit abbilden wollen.

In [ ]:
plot_m4l_dvmc(
    data = [electrons, muons],
    sim = [electrons_by_process, muons_by_process]
)

In **blau** sehen wir die Simulation unserer Hintergrundevents. Wir können erkennen, dass sie unseren Z-Peak sehr gut beschreibt, aber auch den kontinuierlichen Untergrund rechts des Z-Peaks. In **orange** sehen wir unser Signal bei der erwarteten Masse von 125 GeV. Hier erkennen wir, dass sich die Datenpunkte sogar leicht über der Signalerwartung befinden. Wenn wir beachten, dass auch Simulationen ebenfalls Unsicherheiten haben (die wir aus Zeit- und Komplexitätsgründen hier nicht weiter behandeln), dann lässt sich im Allgemeinen eine **gute** Übereinstimmung zwischen den aufgenommenen Daten und den Simulationen feststellen. Das Higgs Boson des Standardmodel ist damit in der Lage den Peak bei 125 GeV zu erklären.

Da wir nun auch die Simulationen haben, können wir ein weiteres Mal, diesmal etwas quantitativer über die statistische Einordnung dieses Peaks sprechen. Wie wir nun sehen, gehört nicht nur der Datenpunkt mit 7 Einträgen zum Higgs-Peak, sondern auch der Datenpunkt daneben. Laut des Standardmodels würden wir in beiden Bins ca. 2 Hintergrundevents erwarten. Wir zählen jedoch 4 bzw. 7 Events in den Daten. Addieren wir beide Bins zusammen und subtrahieren den Hintergrund, dann erhalten wir:
$$ S = N_1 + N_2 - (B_1 + B_2) = 4 + 7 - (2 + 2) = 7 $$
Events Überschuß. Dieser Überschuß hat eine statistische Signifikanz von
$$ \sigma = \frac{S}{\sqrt{B}} = \frac{7}{\sqrt{4}} = 3.5 \ .$$
Die Wahrscheinlichkeit, dass es sich um eine statistische Fluktuation handelt liegt damit bei ca. 0.05\% - entsprechend einem Münzwurf, der **11 mal** hintereinander auf Zahl landet.

> **Good to know:** Da ein ähnlicher Überschuss bei der gleichen Masse im $H\to\gamma\gamma$ Kanal festgestellt wurde konnte man die Ergebnisse kombinieren und eine höhere statistische Signifikanz erreichen. Zusammen mit weiteren Kanälen konnte man die $5\sigma$-Marke knacken.


<font color="red"> **Wichtig:** Die statistische Analyse der Ergebnisse haben wir stark vereinfacht damit sie leicht verstanden werden kann. Bitte beachtet, dass eine rigorose Analyse viele weitere Faktoren beinhaltet und dadurch auch deutlich komplizierter, aber selbstverständlich auch akkurater und konsistenter ist. </font>